# 평가

### 분류모델 평가

### 정확도 (Accuracy)
- 전체 샘플 중에서 올바르게 예측한 샘플의 비율
- 데이터가 불균형한 경우 정확도는 비현실적인 성능을 낼 수 있음

In [50]:
from sklearn.base import BaseEstimator
import numpy as np

# 성별로만 판별하는 모델 작성
class MyTitanicClassifier(BaseEstimator):
    def fit(self,X,y):
        # 훈련 메소드
        pass

    def predict(self,X):
        pred=np.zeros((X.shape[0],1))
        for i in range(X.shape[0]):
            sex=X['Sex'].iloc[i]
            if sex==0:      # 여성
                pred[i]=1   # 생존
        return pred


In [51]:
# 전처리 -> 함수
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

def fillna(df):
    """
    결측치 처리 함수
    - Age : 평균치로 대체
    - Cabin : 'N' 기본값으로 대체
    - Embarked : 'N' 기본값으로 대체
    """

    df['Age']=df['Age'].fillna(df['Age'].mean())
    df['Cabin']=df['Cabin'].fillna('N')
    df['Embarked']=df['Embarked'].fillna('N')

    return df

def drop_feature(df):
    """
    모델 훈련과 관련 없는 속성 제거
    - PassengerId, Name, Ticket
    """
    df=df.drop(['PassengerId','Name','Ticket'],axis=1)

    return df



def encode_feature(df):
    """
    범주형 데이터를 숫자로 인코딩
    - Sex, Cabin, Embarked
    """
    le = LabelEncoder()

    categories=['Sex','Cabin','Embarked']

    for category in categories:
        df[category]=le.fit_transform(df[category])

    return df

    
def scaled_feature(train_data, test_data):
    """
    학습데이터, 테스트데이터
    표준화 후 정규화 
    """

    std_scaler = StandardScaler()
    train_data = std_scaler.fit_transform(train_data)
    test_data = std_scaler.fit_transform(test_data)

    min_max_scaler = MinMaxScaler()
    train_data = min_max_scaler.fit_transform(train_data)
    test_data = min_max_scaler.transform(test_data)

    return train_data,test_data
    

def preprocess_data(df):
    df=fillna(df)
    df=drop_feature(df)
    df=encode_feature(df)

    return df
    


In [52]:
import pandas as pd
from sklearn.model_selection import train_test_split

df=pd.read_csv('./data/titanic.csv')

# 입력/라벨 데이터 분리
X=df.drop('Survived', axis=1)
y=df['Survived']

# 전처리
X=preprocess_data(X)

# 훈련/테스트 데이터 분리
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0)

In [53]:
from sklearn.metrics import accuracy_score
# 모델 훈련
my_classifier=MyTitanicClassifier()
my_classifier.fit(X_train,y_train)

# 예측
pred_train=my_classifier.predict(X_train)
pred_test=my_classifier.predict(X_test)

# 평가 (accuracy_score 사용)
print('훈련 데이터 정확도',accuracy_score(y_train,pred_train))
print('평가 데이터 정확도',accuracy_score(y_test,pred_test))

훈련 데이터 정확도 0.7889221556886228
평가 데이터 정확도 0.7802690582959642


### 혼동행렬 (Confusion Matrix)

![](https://d.pr/i/rtYBJv+)

In [54]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score

- 정밀도 (Precision)
    - 양성이라고 예측한 것(TP + FP) 중에 실제 양성(TP)일 확률
    - 정밀도가 중요한 지표인 경우: 음성인 데이터를 양성으로 예측하면 안되는 경우 (스팸메일 분류 등)

In [55]:
matrix=confusion_matrix(y_test,pred_test)
matrix

array([[115,  24],
       [ 25,  59]])

In [56]:
p_score=59/(24+59)
p_score,precision_score(y_test,pred_test)

(0.7108433734939759, 0.7108433734939759)

- 재현율
    - 실제 양성(TP + FP) 중에 실제 양성으로 예측(TP)한 확률
    - 재현율가 중요한 지표인 경우: 양성인 데이터를 음성으로 예측하면 안되는 경우 (암 진단, 보험/금융 사기 등)

In [57]:
recall_score(y_test,pred_test)

0.7023809523809523

In [58]:
# 잘못 학습된 모델 만들어보기(2)
class MyDeathClassifier(BaseEstimator):
    def fit(self,X,y):
        pass
    
    def predict(self,X):
        return np.zeros((X.shape[0],1)) # 전부 사망
    
my_classifier=MyDeathClassifier()
my_classifier.fit(X_train,y_train)

pred_train=my_classifier.predict(X_train)
pred_test=my_classifier.predict(X_test)

print('훈련 데이터 혼동행렬:\n',confusion_matrix(y_train,pred_train))
print('훈련 데이터 정확도:\n',accuracy_score(y_train,pred_train))
print('훈련 데이터 정밀도:\n',precision_score(y_train,pred_train))
print('훈련 데이터 재현율:\n',recall_score(y_train,pred_train))

print('평가 데이터 혼동행렬:\n',confusion_matrix(y_test,pred_test))
print('평가 데이터 정확도:\n',accuracy_score(y_test,pred_test))
print('평가 데이터 정밀도:\n',precision_score(y_test,pred_test))
print('평가 데이터 재현율:\n',recall_score(y_test,pred_test))

훈련 데이터 혼동행렬:
 [[410   0]
 [258   0]]
훈련 데이터 정확도:
 0.6137724550898204
훈련 데이터 정밀도:
 0.0
훈련 데이터 재현율:
 0.0
평가 데이터 혼동행렬:
 [[139   0]
 [ 84   0]]
평가 데이터 정확도:
 0.6233183856502242
평가 데이터 정밀도:
 0.0
평가 데이터 재현율:
 0.0


c:\Users\Playdata\anaconda3\envs\pystudy_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Playdata\anaconda3\envs\pystudy_env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [59]:
# 오차행렬, 정확도, 정밀도, 재현율 계산 및 출력 함수
def evaluate_binary_classfication(y_true,y_pred):
    print('혼동행렬:\n',confusion_matrix(y_true,y_pred))
    print(f'정확도:{accuracy_score(y_true,y_pred)}, 정밀도:{precision_score(y_true,y_pred)},재현률: {recall_score(y_true,y_pred)}')

In [60]:
from sklearn.linear_model import LogisticRegression    
df=pd.read_csv('./data/titanic.csv')


# 입력/라벨 데이터 분리
X=df.drop('Survived', axis=1)
y=df['Survived']

# 전처리
X=preprocess_data(X)
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0)

# 모델훈련
lr_clf=LogisticRegression()
lr_clf.fit(X_train,y_train)

# 예측 및 평가
pred_train=lr_clf.predict(X_train)
pred_test=lr_clf.predict(X_test)
evaluate_binary_classfication(y_train,pred_train)
evaluate_binary_classfication(y_test,pred_test)


혼동행렬:
 [[354  56]
 [ 79 179]]
정확도:0.7979041916167665, 정밀도:0.7617021276595745,재현률: 0.6937984496124031
혼동행렬:
 [[115  24]
 [ 26  58]]
정확도:0.7757847533632287, 정밀도:0.7073170731707317,재현률: 0.6904761904761905


c:\Users\Playdata\anaconda3\envs\pystudy_env\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


- 정밀도-재현율 trade-off
    - 분류 결정 임계치(threshold)를 낮추면? Positive로 예측할 확률이 높아진다!
        - 정밀도는 낮아지고, 재현율이 높아진다
    - 분류 결정 임계치(threshold)를 높히면? Positive로 예측할 확률이 낮아진다!
        - 정밀도는 높아지고, 재현율이 낮아진다

In [61]:
pred_proba = lr_clf.predict_proba(X_test)
print(pred_proba[:7])

pred = lr_clf.predict(X_test)
print(pred[:7])

[[0.87621241 0.12378759]
 [0.86550613 0.13449387]
 [0.9437167  0.0562833 ]
 [0.09197881 0.90802119]
 [0.2780804  0.7219196 ]
 [0.68484637 0.31515363]
 [0.064489   0.935511  ]]
[0 0 0 1 1 0 1]


In [62]:
from sklearn.preprocessing import Binarizer

temp_X=[[1,-1,2],[2,0,0],[0,1.1,1.2]]
binarizer=Binarizer(threshold=0)
adj_X=binarizer.fit_transform(temp_X)
adj_X

array([[1., 0., 1.],
       [1., 0., 0.],
       [0., 1., 1.]])

In [63]:
from sklearn.preprocessing import Binarizer

# 1(생존)일 확률만 가져오고 + 배치 차원 추가
predict_proba_1=pred_proba[:,1].reshape(-1,1)

binarizer = Binarizer(threshold=0.5)
custom_pred=binarizer.fit_transform(predict_proba_1)
evaluate_binary_classfication(y_test,custom_pred)

binarizer = Binarizer(threshold=0.6)
custom_pred=binarizer.fit_transform(predict_proba_1)
evaluate_binary_classfication(y_test,custom_pred)

혼동행렬:
 [[115  24]
 [ 26  58]]
정확도:0.7757847533632287, 정밀도:0.7073170731707317,재현률: 0.6904761904761905
혼동행렬:
 [[117  22]
 [ 28  56]]
정확도:0.7757847533632287, 정밀도:0.717948717948718,재현률: 0.6666666666666666
